Input bids and subject info

### Notes
- sub-000, ses-20231004OPCoil, 1_attention, 7T
    - 14 off, 240 s stim-on
    - TR=2.539s
    - Q1: f1=0.1 / f2=0.125
- sub-021, ses-VasoTest, 1_frequency_tagging, 7T
    - 14 off, 240 s stim-on
    - TR=3.106
    - Q1: f1=0.1 / f2=0.125
    - 31 runs, discard first run
    - Stimulated frequencies: .14, .22, .1, .15

In [ ]:
import sys
sys.path.append("ComputeCanada/vaso_preproc")

from pathlib import Path
import nibabel as nib

from vaso import sort_vaso_data

project_id = '1_frequency_tagging'
mri_id = '7T'
sub_id = '021'
ses_id = 'VasoTest'
updated_TR = 3.106
# Remove runs depending on volume count
task_id = "wbpilot"
expected_volumes = [82,]

Relabel and group bold and vaso counterparts together
 - vaso sequence produces 2 runs for each scan (blood-nulled and not-nulled contrast)
 - update TR to appropriate value

In [ ]:
vaso_niftis = !ls /data/{project_id}/{mri_id}/bids/sub-{sub_id}/ses-{ses_id}/func/*vaso.*
vaso_niftis = [Path(i) for i in vaso_niftis]

sort_vaso_data(vaso_niftis, updated_TR)

Remove runs depending on volume count

In [ ]:
vaso_niftis = !ls /data/{project_id}/{mri_id}/bids/sub-{sub_id}/ses-{ses_id}/func/*task-{task_id}*.nii.gz
vaso_niftis = [Path(i) for i in vaso_niftis]
run_ids = []
for f in vaso_niftis:
    n_vols = nib.load(f).shape[-1]
    if n_vols not in expected_volumes:
        run_ids.append(f.stem.split("run-")[1].split('_')[0])
run_ids = list(set(run_ids))

for run_id in run_ids:
    print(f"Removing run-{run_id}")
    !rm /data/{project_id}/{mri_id}/bids/sub-{sub_id}/ses-{ses_id}/func/*task-{task_id}*run-{run_id}*.nii.gz
    !rm /data/{project_id}/{mri_id}/bids/sub-{sub_id}/ses-{ses_id}/func/*task-{task_id}*run-{run_id}*.json

Resample task-wholebrain to the same resolution as the vaso data

In [ ]:
wholebrains = !ls  /data/{project_id}/{mri_id}/bids/sub-{sub_id}/ses-{ses_id}/func/*task-wholebrain*part-mag*sbref.nii.gz
for i in wholebrains:
    !echo flirt -in {i} -ref {i} -applyisoxfm 0.8 -out {i}
    !flirt -in {i} -ref {i} -applyisoxfm 0.8 -out {i}

Motion correction stuff

In [ ]:
import numpy as np
TR = 3.106
truncate = (14+25, 14+230)
truncate_idx = [int(i/TR) + 1 for i in truncate]
print(truncate_idx)
n_vols = 82
np.arange(0, TR*n_vols, TR)[truncate_idx[0]:truncate_idx[1]]

In [ ]:
def pca_denoise(bold_path, n_components=10, outfile="mppca.nii.gz"):
    
    import nibabel as nib
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler

    img = nib.load(bold_path)
    data = img.get_fdata()
    x, y, z, n_tps = data.shape
    data_reshaped = data.reshape(-1, n_tps)
    # standardize the data
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_reshaped)
    # Apply MPPCA using PCA
    num_components = n_components  # Number of principal components
    pca = PCA(n_components=num_components)
    data_mppca = pca.fit_transform(data_scaled)
    # Inverse transform to reconstruct the data
    data_reconstructed = pca.inverse_transform(data_mppca)
    data_reconstructed = scaler.inverse_transform(data_reconstructed)
    # Reshape the reconstructed data back to 4D
    data_reconstructed = data_reconstructed.reshape(x, y, z, n_tps)

    nib.save(
        nib.Nifti1Image(data_reconstructed, affine=img.affine, header=img.header),
        outfile,
    )

In [ ]:
outdir = Path("./ComputeCanada/vaso_preproc/tmp")
if not outdir.exists():
    outdir.mkdir(exist_ok=True, parents=True)

vaso_dump = Path("/scratch/fastfmri_playgrond/vaso_dump")
if not vaso_dump.exists():
    vaso_dump.mkdir(exist_ok=True, parents=True)

In [ ]:
BOLDS = !ls  /data/{project_id}/{mri_id}/bids/sub-{sub_id}/ses-{ses_id}/func/*task-wbpilot*part-mag_bold.nii.gz
n_runs = len(BOLDS)
for i in BOLDS:
    print(f" - {Path(i).stem}")

In [ ]:
no_nordic_bold_average = vaso_dump / "average_bold.nii.gz"
nordic_bold_average = vaso_dump / "nordic_average_bold.nii.gz"
no_nordic_vaso_average = vaso_dump / "average_vaso.nii.gz"
nordic_vaso_average = vaso_dump / "nordic_average_vaso.nii.gz"

for bold_ix, bold_orig in enumerate(BOLDS):

    bold_base = Path(bold_orig).stem.replace('.nii','')
    print(bold_ix, bold_base)
    mppca = outdir / f"{bold_base}.mppca.nii.gz"
    pca_denoise(bold_orig, outfile=mppca)

    if bold_ix == 0:
        split = outdir / "split"
        !fslsplit {mppca} {split} -t
        reference = outdir / "split0001.nii.gz"
    
    assert reference.exists(), f"{reference} does not exist."

    # Motion correction
    bold_hmc = outdir / f"{bold_base}.hmc.nii.gz"
    matrix_save = outdir / f"{bold_base}.1dmatrix_save.aff12.1D"
    dfile = outdir / f"{bold_base}.dfile"
    mot = outdir / f"{bold_base}.1Dfile"
    # HMC on MPPCA denoised BOLD
    !3dvolreg \
        -overwrite \
        -prefix {bold_hmc} \
        -base {reference} \
        -1Dmatrix_save {matrix_save} -dfile {dfile} -1Dfile {mot} \
        -quitic \
        {mppca}
    # Apply HMC params to original BOLD data
    !3dAllineate \
        -overwrite \
        -source {bold_orig} \
        -1Dmatrix_apply {matrix_save} \
        -prefix {bold_hmc} 
    # Apply HMC params to NORDIC BOLD data
    bold_nordic = Path(bold_orig.replace('part-mag_', 'part-mag_proc-NORDIC_'))
    assert bold_nordic.exists(), f"{bold_nordic} does not exist."
    bold_nordic_hmc = outdir / f"{bold_base}.nordic.hmc.nii.gz"
    !3dAllineate \
        -overwrite \
        -source {bold_nordic} \
        -1Dmatrix_apply {matrix_save} \
        -prefix {bold_nordic_hmc}
    # Apply HMC params to vaso data
    vaso_orig = Path(bold_orig.replace("bold.nii.gz", "vaso.nii.gz"))
    assert vaso_orig.exists(), f"{vaso_orig} does not exist."
    vaso_hmc = outdir / f"{bold_base}.vaso.hmc.nii.gz"
    !3dAllineate \
        -overwrite \
        -source {vaso_orig} \
        -1Dmatrix_apply {matrix_save} \
        -prefix {vaso_hmc}
    # Apply HMC params to NORDIC vaso data
    vaso_nordic = Path(bold_orig.replace("part-mag_bold", "part-mag_proc-NORDIC_vaso"))
    assert vaso_nordic.exists(), f"{vaso_nordic} does not exist."
    vaso_nordic_hmc = outdir / f"{bold_base}.vaso.nordic.hmc.nii.gz"
    !3dAllineate \
        -overwrite \
        -source {vaso_nordic} \
        -1Dmatrix_apply {matrix_save} \
        -prefix {vaso_nordic_hmc}


    # Highpass filter
    bold_hmc_hpf = outdir / f"{bold_base}.hmc.hpf.nii.gz"
    bold_nordic_hmc_hpf = outdir / f"{bold_base}.nordic.hmc.hpf.nii.gz"
    vaso_hmc_hpf = outdir / f"{bold_base}.vaso.hmc.hpf.nii.gz"
    vaso_nordic_hmc_hpf = outdir / f"{bold_base}.vaso.nordic.hmc.hpf.nii.gz"
    tmean = outdir / "tmean.nii.gz"

    for _hmc, _hmc_hpf in zip(
        [bold_hmc, bold_nordic_hmc, vaso_hmc, vaso_nordic_hmc], 
        [bold_hmc_hpf, bold_nordic_hmc_hpf, vaso_hmc_hpf, vaso_nordic_hmc_hpf]
    ):
        !fslmaths {_hmc} -Tmean {tmean}
        !3dBandpass -overwrite -prefix {_hmc_hpf} -dt {TR} .01 9999 {_hmc}
        !fslmaths {_hmc_hpf} -add {tmean} {_hmc_hpf}
        assert _hmc_hpf.exists()

    if bold_ix == 0:
        !cp {bold_hmc_hpf} {no_nordic_bold_average}
        !cp {bold_nordic_hmc_hpf} {nordic_bold_average}
        !cp {vaso_hmc_hpf} {no_nordic_vaso_average}
        !cp {vaso_nordic_hmc_hpf} {nordic_vaso_average}
    else:
        !fslmaths {no_nordic_bold_average} -add {bold_hmc_hpf} {no_nordic_bold_average}
        !fslmaths {nordic_bold_average} -add {bold_nordic_hmc_hpf} {nordic_bold_average}
        !fslmaths {no_nordic_vaso_average} -add {vaso_hmc_hpf} {no_nordic_vaso_average}
        !fslmaths {nordic_vaso_average} -add {vaso_nordic_hmc_hpf} {nordic_vaso_average}

!fslmaths {no_nordic_bold_average} -div {n_runs} {no_nordic_bold_average}
!fslmaths {nordic_bold_average} -div {n_runs} {nordic_bold_average}
!fslmaths {no_nordic_vaso_average} -div {n_runs} {no_nordic_vaso_average}
!fslmaths {nordic_vaso_average} -div {n_runs} {nordic_vaso_average}
!fslmaths {no_nordic_vaso_average} -div {no_nordic_bold_average} {no_nordic_vaso_average}
!fslmaths {nordic_vaso_average} -div {nordic_bold_average} {nordic_vaso_average}

In [ ]:
import statsmodels.api as sm
import itertools

no_nordic_bold_average = vaso_dump / "average_bold.nii.gz"
nordic_bold_average = vaso_dump / "nordic_average_bold.nii.gz"
no_nordic_vaso_average = vaso_dump / "average_vaso.nii.gz"
nordic_vaso_average = vaso_dump / "nordic_average_vaso.nii.gz"

search_frequencies = [.14, .22, .1, .15, .08, .05, .12]
for _nifti in [no_nordic_bold_average, no_nordic_vaso_average]:
    for search_frequency in search_frequencies:
        
        metric_out = Path(str(_nifti).replace('.nii.gz',f'_f-{search_frequency}_r2.nii.gz'))
        if metric_out.exists():
            print(f"{metric_out.stem} already generated.\nSkipping.")
            continue
        else:
            print(f"Generating metric: {metric_out.stem}")
        
        # load nifti
        img = nib.load(_nifti)
        ts_data = img.get_fdata()[:,:,:,truncate_idx[0]:truncate_idx[1]]
        x, y, z, n_tps = ts_data.shape
        # Run GLM on each vertex
        r2_data = np.zeros((x,y,z))
        voxel_coordinates = itertools.product(range(x), range(y), range(z))
    
        n_voxels = np.prod((x,y,z))
        # Iterate over the voxel coordinates
        for voxel_idx, coordinates in enumerate(voxel_coordinates):
            # Process the voxel coordinates (x, y, z) here
            _x, _y, _z = coordinates
    
            if voxel_idx % 50_000 == 0:
                print(f"[PROGRESS] {str(voxel_idx).zfill(7)}/{str(n_voxels).zfill(7)}")
    
            # Get timeseries from a voxel, and associated timepoints
            y = ts_data[_x, _y, _z, :]
            # Skip if all values is zero
            if np.all(y == 0):
                continue
                
            t = np.linspace(0, TR*n_tps, n_tps+1)[:-1] # Non-phased timepoints
            t = np.fmod(t, 1/search_frequency) # Phased timepoints

            # GLM - fit phased
            X = np.vstack((np.sin(2*np.pi*t*search_frequency), np.cos(2*np.pi*t*search_frequency))).T
            X = sm.add_constant(X)
            model = sm.GLM(y, X, family=sm.families.Gaussian())
            try:
                result = model.fit()
                # Calculate R2
                y_pred = result.predict(X)
                y_mean = np.mean(y)
                ss_total = np.sum((y - y_mean) ** 2)  # Total sum of squares
                ss_residual = np.sum((y - y_pred) ** 2)  # Residual sum of squares
                r2 = 1 - (ss_residual / ss_total)  # R-squared
                # store r2
                r2_data[_x, _y, _z] = r2
            except:
                r2_data[_x, _y, _z] = -1
    
        # Save as dscalar
        r2_img = nib.Nifti1Image(r2_data, header = img.header, affine = img.affine)
        nib.save(r2_img, metric_out)